##### LLMs are extremely powerful models, but they have a problem with data recency and hallucinations. To reduce this, we use vectara... Vectara provides an end-to-end managed service for Retrieval Augmented Generation or RAG.

#####In this note book we made RAG based Q/A nutrition web app where user can ask question related to diet plan.. For this we use vectara alnog with langchain.






#Installing required dependencies
First we will install all the required extrnal libraries as given below:




1.   <b> langchain:</b> LangChain is a framework that provides to connect language models with external data sources.
2.   <b>pypdf</b>: a Python library for working with PDF files.
3.   <b>Transformers:</b> library is a popular open-source library by Hugging Face that provides pre-trained models.
4.   <b>entence-transformers:</b> sentence-transformers library is a Python package that allows you to easily use pre-trained sentence embeddings models


<b>One thing to remember is that after installing libraries you have to restart the run time again so that other dependencies are not affected by it.</b>

In [ ]:
!pip install langchain
!pip install pypdf
!pip install Transformers
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 811.8/811.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.4/239.4 kB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.4/55.4 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.8/132.8 kB 3.9 MB/s eta 0:00:00


#Importing required dependencies

In [ ]:
from langchain import HuggingFaceHub
from transformers import AutoModelForCausalLM
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import vectara
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_community.vectorstores import Vectara
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader
from langchain_community.embeddings.fake import FakeEmbeddings
from getpass import getpass

#Vectara
Vectara provides an end-to-end managed service for Retrieval Augmented Generation or RAG. To get started with Vectara, we went to https://console.vectara.com/signup or click Get Started Free at vectara.com. After we made an account,  create our first corpus and upload our document!. we extract three IDs after that:


1.   Vectara Corpus ID
2.   Vectara Customer ID
3.   Vectara API-Key









In [ ]:
#right here your actual vectra_corpus_id, vectra_customer_id,and vectra_API_Kay
vectara_corpus_id = os.environ['VECTARA_CORPUS_ID']
vectara_customer_id = os.environ['VECTARA_CUSTOMER_ID']
vectara_api_key = os.environ['VECTARA_API_KEY']

In [ ]:

import os
os.environ['VECTARA_API_KEY'] = 'VECTARA_CORPUS_ID'
os.environ['VECTARA_CORPUS_ID'] = 'VECTARA_CUSTOMER_ID'
os.environ['VECTARA_CUSTOMER_ID']='VECTARA_API_KEY'

#Load Document:
load the pdf document. We integrate the langchain with vectara ground generation to upload document and store in vectra store...  

In [ ]:
loader = DirectoryLoader("sample_data/folder", glob = "./*.pdf", loader_cls = PyPDFLoader)
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

###Large Language Mode (Mistral-7b)
We use Mistaral LLM model from hugging-face...

In [ ]:
HUGGINGFACEHUB_API_TOKEN = getpass()

··········


In [ ]:
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [ ]:
repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
llm = HuggingFaceHub(repo_id = repo_id, model_kwargs= {"temperature": 0.5, "max_length": 10000,"max_new_tokens": 3000})

In [ ]:
vectorstore = Vectara(
                vectara_customer_id=vectara_customer_id,
                vectara_corpus_id=vectara_corpus_id,
                vectara_api_key=vectara_api_key
            )

In [ ]:
vectara = Vectara.from_documents(
    docs,
    embedding=None, #we use default vectara embedding
)


In [ ]:
starting_tag = "<s>[INST]"
ending_tag = "</s>[INST]"
prompt_template = """You are an AI assisstant which helps the people to balance their diet by prvoding them diet plans.
                      You need to provide the diet plans. which should include breakfast, lunch and dinner
                  Context :{context}
                  Question: {question}"""

prompt = starting_tag + prompt_template + ending_tag
PROMPT = PromptTemplate(template=prompt, input_variables=["context","question"])
chain = RetrievalQA.from_chain_type(
        llm = llm,
        chain_type = "stuff",
        retriever = vectara.as_retriever(),
        input_key = "question",
        chain_type_kwargs={"prompt": PROMPT},
        return_source_documents = False
    )

### Query
User can ask diet related questions and get the appropriate answers with less hallucinated answer.

In [ ]:
response = chain(input("Any diet questions or thoughts you'd like to share?  "))
if "result" in response:
  result = response["result"]
  parts = result.split("</s>[INST]")
  if len(parts) > 1:

    answer = parts[1].strip()
    print(answer)
  else:
    print("Not Found")
# response

Any diet questions or thoughts you'd like to share?  give the 25 years girl diet plan
Based on the information provided in the context, the diet plan for a 25-year-old woman according to the Irish food-based dietary guidelines would include:

1. Grains: At least 5 servings per day, preferably whole grains. One serving equals 30g of cooked cereals, 1 medium slice of bread, or 1 medium portion of pasta or rice.
2. Fruit and vegetables: At least 7 portions per day. One portion equals approximately 80g of raw fruit or vegetables.
3. Protein foods: At least 2 servings of protein-rich foods per day from the following sources: meat, poultry, fish, eggs, beans, pulses, nuts, and seeds. One serving equals approximately 80g cooked meat, 100g cooked fish, 2 eggs, or 100g cooked beans.
4. Dairy: At least 3 servings per day. One serving equals approximately 200ml of milk, 100g of cheese, or 150ml of yogurt.
5. Oil and spreads: Use sparingly. One serving equals approximately 5g (1 teaspoon) of unsat

#Hallucination Model Score
we comapre the generated vectra base LLM query scores using HHEM model from huggingface..

In [ ]:
from sentence_transformers import CrossEncoder
import numpy as np
np.set_printoptions(suppress=True, precision=4)

model = CrossEncoder('vectara/hallucination_evaluation_model')

example_pairs = [
    # diet plan of 30 years old man
{"LLM_vectara":"""**Breakfast:**
- Whole grain cereal or porridge with low-fat milk and fresh fruit (banana, berries, or apple)- A small portion of lean protein such as boiled or poached eggs, or a handful of nuts (almonds, walnuts, or cashews)

**Mid-Morning Snack:**
- A piece of fruit (apple, banana, orange, or pear) or a handful of raw vegetables (carrots, cucumber, or bell peppers) with hummus or salsa

**Lunch:**
- A balanced meal with lean protein (grilled chicken, fish, or turkey), complex carbohydrates (quinoa, brown rice, or whole grain bread), and plenty of vegetables (spinach, tomatoes, broccoli, or carrots)

**Afternoon Snack:**
- A small serving of nuts and seeds (pumpkin seeds, sunflower seeds, or a handful of mixed nuts) or a piece of fruit

**Dinner:**
- A balanced meal with lean protein, complex carbohydrates, and plenty of vegetables. Consider incorporating iron-rich foods (lean red meat, lentils, or spinach) and sources of calcium, vitamin D, and folic acid (broccoli, fortified cereals, or almonds)

**Evening Snack:**
- A small serving of fruit or a handful of raw vegetables with hummus or salsa

Remember to monitor portion sizes to maintain a healthy weight and avoid excessive calorie intake. Adjust the plan according to your personal preferences and dietary needs. This diet plan is based on the Irish food pyramid and the National Food-based Dietary Guidelines.",
     "LLM response": "The woman is playing a game resting.""",

"LLM":"""Breakfast:

Whole grain cereal or oats with milk or a dairy-free alternative
Fresh fruits like berries or banana
A source of protein (e.g., Greek yogurt, eggs, or nut butter)
Mid-Morning Snack:

A piece of fruit (e.g., apple or pear)
Handful of nuts or seeds
Lunch:

Grilled chicken, fish, tofu, or legumes for protein
Quinoa, brown rice, or sweet potatoes for complex carbohydrates
Vegetables (colorful variety) for fiber and vitamins
Afternoon Snack:

Greek yogurt with a drizzle of honey
Carrot or cucumber sticks with hummus
Dinner:

Lean protein (chicken, turkey, fish, or plant-based protein)
Steamed or roasted vegetables
Whole grains (brown rice, quinoa, or whole wheat pasta)
Evening Snack (if hungry):

A small serving of cottage cheese with pineapple or a handful of cherry tomatoes
Hydration:

Drink plenty of water throughout the day (at least 8 glasses)
Limit sugary beverages and opt for herbal teas or infused water
General Tips:

Control portion sizes to avoid overeating.
Limit processed foods, sugary snacks, and high-fat items.
Aim for a colorful plate with a variety of fruits and vegetables.
Consider consulting with a nutritionist or healthcare professional for personalized advice.
Remember, it's essential to listen to your body's hunger and fullness cues and make adjustments based on your specific needs and preferences. Additionally, if you have any existing health conditions or dietary restrictions, it's advisable to consult with a healthcare professional or nutritionist for tailored guidance.  """


    },

# diet plan of 50 years old man
{"LLM_vectara": """Breakfast:
- Oatmeal made with water or unsweetened almond milk, topped with berries, walnuts, and a drizzle of honey
- A slice of whole grain toast with avocado or peanut butter

Lunch:
- Grilled chicken or fish, served with a side of quinoa or brown rice, and steamed vegetables such as broccoli, carrots, or bell peppers
- A small salad with mixed greens, tomatoes, cucumber, and a vinaigrette dressing

Dinner:
- Grilled or baked lean meat (beef, chicken, or fish), served with a side of steamed vegetables and a small portion of sweet potato or whole grain pasta
- A side salad with mixed greens, tomatoes, cucumber, and a vinaigrette dressing

Snacks:
- A handful of mixed nuts or seeds
- A piece of fruit or a small apple with a handful of almonds or peanuts

This diet plan provides a good balance of heart-healthy foods, including lean proteins, whole grains, fruits, vegetables, and healthy fats. It also includes omega-3 fatty acids from sources such as walnuts, salmon, and flaxseeds, which are important for heart health. Additionally, it is low in sodium and saturated fat, which are risk factors for heart disease.""",
"LLM": """Breakfast:

Whole grain toast or oatmeal
Eggs or egg whites
Low-fat dairy or a dairy alternative
Berries or a piece of fruit
Mid-Morning Snack:

Greek yogurt or a small handful of nuts
A piece of fruit, like an apple or a banana
Lunch:

Grilled chicken, fish, or lean meat
Quinoa, brown rice, or whole wheat pasta
Leafy green salad or a variety of colorful vegetables
Olive oil or a light vinaigrette dressing
Afternoon Snack:

Hummus with vegetable sticks (carrots, cucumber)
A small handful of almonds or walnuts
Dinner:

Baked or grilled fish, chicken, or tofu
Steamed or roasted vegetables (broccoli, Brussels sprouts, sweet potatoes)
Quinoa, barley, or another whole grain
A side salad with a variety of greens
Evening Snack (if hungry):

Low-fat cottage cheese with fresh pineapple or a small bowl of berries
Hydration:

Drink plenty of water throughout the day (at least 8 glasses)
Include herbal teas or infused water for variety
General Tips:

Aim for a balanced plate with a mix of lean protein, whole grains, and colorful vegetables.
Watch portion sizes to manage calorie intake.
Limit added sugars, processed foods, and high-fat items.
Include foods rich in calcium and vitamin D for bone health, such as dairy, fortified plant-based milk, or leafy greens.
Consider incorporating heart-healthy fats, such as avocados, nuts, and olive oil.
Again, individual nutritional needs can vary, and this is a general guideline. Consultation with a healthcare professional or a registered dietitian can help create a personalized plan based on specific health conditions, dietary preferences, and lifestyle factors."""},

{"LLM_vectara": """ Breakfast:
- 1 medium slice of whole grain bread (1 serving of grains)
- 1 banana (1 portion of fruit)
- 1 boiled egg (1 serving of protein foods)

Mid-morning snack:
- 1 medium apple (1 portion of fruit)
- 1 small handful of mixed nuts (1 serving of protein foods or oil and spreads)

Lunch:
- 100g cooked whole grain pasta (2 servings of grains)
- 100g cooked lean chicken breast (1 serving of protein foods)
- 100g steamed broccoli (1 portion of vegetables)

Afternoon snack:
- 1 medium pear (1 portion of fruit)
- 1 small pot of hummus (1 serving of protein foods or oil and spreads)

Dinner:
- 100g cooked salmon (1 serving of protein foods and 1 serving of oily fish)
- 150ml cooked brown rice (1 serving of grains)
- 100g cooked carrots (1 portion of vegetables)
- 150ml low-fat milk (1 serving of dairy)

Dessert (optional):
- 1 medium-sized banana (1 portion of fruit)
- 1 small scoop of low-fat ice cream (1 serving of dairy)

This meal plan provides a good balance of nutrient-dense foods from all the food groups, but remember to adjust portion sizes based on individual needs and preferences. Enjoy your meals and happy cooking! """,
"LLM":""" Breakfast:

Whole grain cereal or oatmeal
Low-fat or plant-based milk
Fresh berries or a banana
A source of protein like Greek yogurt or eggs
Mid-Morning Snack:

Handful of nuts (almonds, walnuts, or pistachios)
A piece of fruit (apple, pear, or berries)
Lunch:

Grilled chicken, tofu, or beans for protein
Quinoa, brown rice, or whole wheat wraps
Mixed vegetables (broccoli, bell peppers, spinach)
Olive oil or a light dressing for flavor
Afternoon Snack:

Greek yogurt with honey or a small bowl of cottage cheese
Carrot or cucumber sticks with hummus
Dinner:

Baked or grilled fish, lean meat, or plant-based protein
Sweet potatoes or whole grain pasta/rice
Steamed or roasted vegetables (asparagus, Brussels sprouts, or zucchini)
A side salad with a variety of colorful veggies
Evening Snack (if hungry):

A small bowl of mixed berries or a piece of dark chocolate
Herbal tea or warm milk with a sprinkle of cinnamon
Hydration:

Drink plenty of water throughout the day (at least 8 glasses)
Include herbal teas or infused water for variety
General Tips:

Aim for a balanced diet with a variety of food groups.
Watch portion sizes to maintain a healthy weight.
Limit processed foods, sugary snacks, and high-fat items.
Include calcium-rich foods like dairy or fortified plant-based milk.
Ensure adequate intake of iron from sources like lean meat, beans, and dark leafy greens. """}

]


scores = model.predict(
    [ [p["LLM_vectara"], p["LLM"]] for p in example_pairs ]
)

scores

array([0.7131, 0.7553, 0.7461], dtype=float32)